In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import warnings
from IPython.display import HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle
import seaborn as sns
import shutil


import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, default_collate, Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

from PIL import Image, ImageFile

from tqdm.auto import tqdm

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
color_list = ['#faa5c7', '#eb679c', '#cc3d76', '#b3245d', '#d40457', '#8f1042', '#6e042e']
color_palette = sns.color_palette(color_list)
sns.set_palette(color_palette)

# cmap = colors.ListedColormap(color_palette)
cmap = LinearSegmentedColormap.from_list('custom_palette', color_list, N=256)

warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
from google.colab import drive
import os
import shutil

# Google Drive'daki giriş ve çalışma dizinleri
input_directory = '/content/drive/MyDrive/FracatlasORJ2'
output_directory = '/content/drive/MyDrive/You2_Working_Directory0'

# Dataset dizini ve alt dizinler
dataset_directory = f'{output_directory}/dataset'
output_train_directory = f'{dataset_directory}/Bone_Fracture_Binary_Classification/test'
output_dev_directory = f'{dataset_directory}/Bone_Fracture_Binary_Classification/train'
output_test_directory = f'{dataset_directory}/Bone_Fracture_Binary_Classification/val'

# Dizinleri oluşturma
os.makedirs(output_train_directory, exist_ok=True)
os.makedirs(output_dev_directory, exist_ok=True)
os.makedirs(output_test_directory, exist_ok=True)

# Verileri kopyalama
shutil.copytree(f'{input_directory}/test', output_train_directory, dirs_exist_ok=True)
shutil.copytree(f'{input_directory}/train', output_dev_directory, dirs_exist_ok=True)
shutil.copytree(f'{input_directory}/val', output_test_directory, dirs_exist_ok=True)


'/content/drive/MyDrive/You2_Working_Directory0/dataset/Bone_Fracture_Binary_Classification/val'

In [ ]:
# Iterate through subdirectories
def removeCorruptFiles(directory_path):
  for item in os.listdir(directory_path):
    item_path = os.path.join(directory_path, item)

    if os.path.isdir(item_path):
      removeCorruptFiles(item_path)
    else:
      try:
        Image.open(item_path).verify()
      except (IOError, SyntaxError):
        os.remove(item_path)
        print(f'Removed corrupted image: {item}')

  # print("Corrupted image removal complete.")

In [ ]:
def removeEmptySubdirectories(directory):
    # Iterate over each entry in the directory
    for entry in os.listdir(directory):
        # Create full path
        full_path = os.path.join(directory, entry)

        # If entry is a directory, recursively remove empty subdirectories
        if os.path.isdir(full_path):
            removeEmptySubdirectories(full_path)

            # Check if directory is empty after removal
            if not os.listdir(full_path):
                # Remove the directory if it's empty
                os.rmdir(full_path)
                print(f"Removed empty directory: {full_path}")

In [ ]:
def copyDatasetIntoWorkingDir(source_directory, destination_directory):
    # Remove the destination directory if it already exists
    if os.path.exists(destination_directory):
        shutil.rmtree(destination_directory)


    shutil.copytree(source_directory, destination_directory)

In [ ]:
IMAGE_SIZE = 224
imgSize = IMAGE_SIZE

In [ ]:
import torchvision
from torchvision import transforms

# Dönüşüm (transforms) tanımlamaları
imgSize = 224  # Örnek bir görüntü boyutu

train_transform = transforms.Compose([
    transforms.Resize((imgSize, imgSize)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

dev_transform = transforms.Compose([
    transforms.Resize((imgSize, imgSize)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

test_transform = transforms.Compose([
    transforms.Resize((imgSize, imgSize)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

# ImageFolder veri kümesi oluşturma
train_dataset = torchvision.datasets.ImageFolder(root=output_train_directory, transform=train_transform)
dev_dataset = torchvision.datasets.ImageFolder(root=output_dev_directory, transform=dev_transform)
test_dataset = torchvision.datasets.ImageFolder(root=output_test_directory, transform=test_transform)


In [ ]:
def my_collate(batch):
    "Puts each data field into a tensor with outer dimension batch size"
    batch = list(filter(lambda x:x is not None, batch))
    return default_collate(batch)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size = batchsize, shuffle = True, collate_fn = my_collate)
dev_dataloader = DataLoader(dev_dataset, batch_size = batchsize, collate_fn = my_collate)
test_dataloader = DataLoader(test_dataset, batch_size = batchsize, collate_fn = my_collate)

In [ ]:

def makeTheNet(k = 3, s = 1, p = 2):
    class vİT(nn.Module):
        def __init__(self):
            super().__init__()

            imgSize = IMAGE_SIZE

            self.conv1 = nn.Conv2d(3, 128, k, s, p)
            self.bnorm1 = nn.BatchNorm2d(128)

            imgSize = ((imgSize - k + 2*p) // s) + 1
            imgSize = imgSize // k

            self.conv2 = nn.Conv2d(128, 256, k, s, p)
            self.bnorm2 = nn.BatchNorm2d(256)

            imgSize = ((imgSize - k + 2*p) // s) + 1
            imgSize = imgSize // k

            self.conv3 = nn.Conv2d(256, 256, k, s, p)
            self.bnorm3 = nn.BatchNorm2d(256)

            imgSize = ((imgSize - k + 2*p) // s) + 1
            imgSize = imgSize // k

            self.fc1 = nn.Linear(imgSize * imgSize * 256, 256)
            self.fc2 = nn.Linear(256, 256)
            self.fc3 = nn.Linear(256, 256)
            self.fc4 = nn.Linear(256, 256)
            self.output = nn.Linear(256, 1)

        def forward(self, x):
            x = F.max_pool2d( self.conv1(x), k)
            x = F.leaky_relu( self.bnorm1(x) )

            x = F.max_pool2d( self.conv2(x), k)
            x = F.leaky_relu( self.bnorm2(x) )

            x = F.max_pool2d( self.conv3(x), k)
            x = F.leaky_relu( self.bnorm3(x) )

            nUnits = x.shape.numel()/x.shape[0]
            x = x.view(-1,int(nUnits))

            x = F.leaky_relu( self.fc1(x) )
            x = F.dropout2d(x, .5)

            x = F.leaky_relu( self.fc2(x) )
            x = F.dropout2d(x, .5)

            x = F.leaky_relu( self.fc3(x) )
            x = F.dropout2d(x, .5)

            x = F.leaky_relu( self.fc4(x) )
            x = F.dropout2d(x, .5)

            return self.output(x)

    net = torchvision.models.vit_b_16(weights = 'DEFAULT')

    for p in net.parameters():
      p.requires_grad = False

    num_features = net.heads.head.in_features  # Get the number of input features to the last layer
    net.heads.head = nn.Linear(num_features, 1)
    lossfun = nn.BCEWithLogitsLoss()

    optimizer = torch.optim.Adam(net.parameters(), lr = .001)

    return net, lossfun, optimizer

In [ ]:
net, lossfun, optimizer = makeTheNet()
net.to(device)
X, y = X.to(device), y.to(device, dtype = torch.float)
yHat = net(X)
yHat = yHat.squeeze()
lossfun(yHat, y)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:03<00:00, 106MB/s]


tensor(0.6776, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [ ]:
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torch.optim.lr_scheduler import StepLR, MultiStepLR, ExponentialLR, CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
import optuna
from optuna.samplers import TPESampler
from torchvision import transforms
import torch
import numpy as np
from tqdm import tqdm

def function2TrainModel(train_dataloader,dev_dataloader,test_dataloader,device,epochs,learning_rate,bs,dropout_prob,optimizer_name,num_layers,scheduler_type):

    model = ViT(num_layers=num_layers).to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    elif optimizer_name == 'AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    if scheduler_type == 'StepLR':
        scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
    elif scheduler_type == 'MultiStepLR':
        scheduler = MultiStepLR(optimizer, milestones=[10, 20], gamma=0.1)
    elif scheduler_type == 'ExponentialLR':
        scheduler = ExponentialLR(optimizer, gamma=0.9)
    elif scheduler_type == 'CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=10)
    elif scheduler_type == 'ReduceLROnPlateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

    train_dataset = ImageDataset(train_data)
    val_dataset = ImageDataset(val_data)
    train_dataloader = DataLoader(train_dataset, num_workers=1, batch_size=bs, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=1, batch_size=bs, shuffle=False)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    val_precisions, val_recalls, val_f1s = [], [], []

    best_val_loss = float('inf')
    best_val_accuracy = 0.0

    for epoch in range(epochs):
        total_acc_train = 0
        total_loss_train = 0.0
        model.train()

        for train_image, train_label in tqdm(train_dataloader):
            train_image, train_label = train_image.to(device), train_label.to(device)
            output = model(train_image)
            loss = criterion(output, train_label)
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc
            total_loss_train += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss_train = total_loss_train / len(train_dataset)
        avg_acc_train = total_acc_train / len(train_dataset)

        total_acc_val = 0
        total_loss_val = 0.0
        all_preds = []
        all_labels = []
        model.eval()

        with torch.no_grad():
            for val_image, val_label in tqdm(val_dataloader):
                val_image, val_label = val_image.to(device), val_label.to(device)
                output = model(val_image)
                loss = criterion(output, val_label)
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
                total_loss_val += loss.item()

                all_preds.extend(output.argmax(dim=1).cpu().numpy())
                all_labels.extend(val_label.cpu().numpy())

        avg_loss_val = total_loss_val / len(val_dataset)
        avg_acc_val = total_acc_val / len(val_dataset)

        train_losses.append(avg_loss_train)
        val_losses.append(avg_loss_val)
        train_accuracies.append(avg_acc_train)
        val_accuracies.append(avg_acc_val)

        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        f1 = f1_score(all_labels, all_preds, average='weighted')

        val_precisions.append(precision)
        val_recalls.append(recall)
        val_f1s.append(f1)

        print(f'Epoch: {epoch + 1} | Train Loss: {avg_loss_train:.3f} | Train Accuracy: {avg_acc_train:.3f} | Val Loss: {avg_loss_val:.3f} | Val Accuracy: {avg_acc_val:.3f}')
        print(f'Precision: {precision:.3f} | Recall: {recall:.3f} | F1 Score: {f1:.3f}')

        if scheduler_type == 'ReduceLROnPlateau':
            scheduler.step(avg_loss_val)
        else:
            scheduler.step()

    return model, avg_acc_val

def evaluate_model(model, test_data):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    test_dataset = ImageDataset(test_data)
    test_dataloader = DataLoader(test_dataset, num_workers=1, batch_size=32, shuffle=False)

    model.eval()
    total_acc_test = 0
    total_loss_test = 0.0
    all_preds = []
    all_labels = []
    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for test_image, test_label in tqdm(test_dataloader):
            test_image, test_label = test_image.to(device), test_label.to(device)
            output = model(test_image)
            loss = criterion(output, test_label)
            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
            total_loss_test += loss.item()

            all_preds.extend(output.argmax(dim=1).cpu().numpy())
            all_labels.extend(test_label.cpu().numpy())

    avg_loss_test = total_loss_test / len(test_dataloader.dataset)
    avg_acc_test = total_acc_test / len(test_dataloader.dataset)

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Test Loss: {avg_loss_test:.3f} | Test Accuracy: {avg_acc_test:.3f}')
    print(f'Test Precision: {precision:.3f} | Test Recall: {recall:.3f} | Test F1 Score: {f1:.3f}')

    return avg_acc_test

def bayesian_optimization(train_data, val_data, test_data, n_trials=50):
    def objective(trial):
        epochs = trial.suggest_int('epochs', 10, 30)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
        batch_size = trial.suggest_int('batch_size', 16, 64)
        dropout_prob = trial.suggest_uniform('dropout_prob', 0.3, 0.7)
        optimizer_name = trial.suggest_categorical('optimizer_name', ['Adam', 'SGD', 'AdamW'])
        num_layers = trial.suggest_int('num_layers', 12, 24)
        scheduler_type = trial.suggest_categorical('scheduler_type', ['StepLR', 'MultiStepLR', 'ExponentialLR', 'CosineAnnealingLR', 'ReduceLROnPlateau'])

        model, val_accuracy = model_train(train_data, val_data, epochs, learning_rate, batch_size, dropout_prob, optimizer_name, num_layers, scheduler_type)

        return val_accuracy

    sampler = TPESampler()
    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=n_trials)

    best_trial = study.best_trial
    print(f'Best trial parameters: {best_trial.params}')

    best_model, _ = model_train(train_data, val_data,
                                best_trial.params['epochs'],
                                best_trial.params['learning_rate'],
                                best_trial.params['batch_size'],
                                best_trial.params['dropout_prob'],
                                best_trial.params['optimizer_name'],
                                best_trial.params['num_layers'],
                                best_trial.params['scheduler_type'])

    return best_model

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

best_model = bayesian_optimization(train_data, val_data, test_data)


evaluate_model(best_model, test_data)


[I 2024-06-10 09:15:42,972] A new study created in memory with name: no-name-15605e60-fe71-4d37-a529-d3f01e200b78
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['encoder.layer.12.attention.attention.key.bias', 'encoder.layer.12.attention.attention.key.weight', 'encoder.layer.12.attention.attention.query.bias', 'encoder.layer.12.attention.attention.query.weight', 'encoder.layer.12.attention.attention.value.bias', 'encoder.layer.12.attention.attention.value.weight', 'encoder.layer.12.attention.output.dense.bias', 'encoder.layer.12.attention.output.dense.weight', 'encoder.layer.12.intermediate.dense.bias', 'encoder.layer.12.intermediate.dense.weight', 'encoder.layer.12.layernorm_after.bias', 'encoder.layer.12.layernorm_after.weight', 'encoder.layer.12.layernorm_before.bias', 'encoder.layer.12.layernorm_before.weight', 'encoder.layer.12.output.dense.bias', 'encoder.layer.12.output.dense.weight', 'enco

Epoch: 1 | Train Loss: 0.013 | Train Accuracy: 0.507 | Val Loss: 0.013 | Val Accuracy: 0.451
Precision: 0.318 | Recall: 0.451 | F1 Score: 0.328


100%|██████████| 12/12 [00:15<00:00,  1.30s/it]


Epoch: 2 | Train Loss: 0.013 | Train Accuracy: 0.508 | Val Loss: 0.013 | Val Accuracy: 0.449
Precision: 0.340 | Recall: 0.449 | F1 Score: 0.336


100%|██████████| 12/12 [00:15<00:00,  1.28s/it]


...

Epoch: 17 | Train Loss: 0.017 | Train Accuracy: 0.652 | Val Loss: 0.018 | Val Accuracy: 0.692
Precision: 0.693 | Recall: 0.692 | F1 Score: 0.692


100%|██████████| 18/18 [00:15<00:00,  1.15it/s]


Epoch: 18 | Train Loss: 0.017 | Train Accuracy: 0.653 | Val Loss: 0.018 | Val Accuracy: 0.702
Precision: 0.703 | Recall: 0.702 | F1 Score: 0.702


 90%|█████████ | 109/121 [04:22<00:28,  2.41s/it]
[W 2024-06-11 07:18:25,703] Trial 16 failed with parameters: {'epochs': 21, 'learning_rate': 5.404840041797893e-05, 'batch_size': 39, 'dropout_prob': 0.520211683966328, 'optimizer_name': 'SGD', 'num_layers': 23, 'scheduler_type': 'ExponentialLR'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-17-9719ab8023e9>", line 198, in objective
    model, val_accuracy = model_train(train_data, val_data, epochs, learning_rate, batch_size, dropout_prob, optimizer_name, num_layers, scheduler_type)
  File "<ipython-input-17-9719ab8023e9>", line 94, in model_train
    train_image, train_label = train_image.to(device), train_label.to(device)
KeyboardInterrupt
[W 2024-06-11 07:18:25,707] Trial 16 failed with value None.


KeyboardInterrupt: 

In [ ]:
# Şimdi test veri kümesinde doğruluk ve kayıp değerlerini hesaplayalım
test_acc = []
test_loss = []

net.eval()
with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device, dtype=torch.float)
        yHat = net(X)
        yHat = yHat.squeeze()
        yHat, y = yHat.cpu(), y.cpu()

        loss = lossfun(yHat, y)
        test_acc.append(torch.mean(((yHat > 0.0) == y).float()) * 100)
        test_loss.append(loss.item())

# Test veri kümesinde elde edilen sonuçları rapor edelim
test_accuracy = np.mean(test_acc)
test_mean_loss = np.mean(test_loss)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(17, 5))
ax[0].plot(trainAcc, 'o-', label='Train')
ax[0].plot(devAcc, 'o-', label='Validation')
ax[0].legend()
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].set_title(f' Validation accuracy {np.mean(devAcc[-10:]):.2f}')

ax[1].plot(trainLoss, 'o-', label='Train')
ax[1].plot(devLoss, 'o-', label='Validation')
ax[1].legend()
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].set_title(f'Validation loss {np.mean(devLoss[-10:]):.2f}')




In [ ]:
import torch
import numpy as np

def evaluateTest(net, test_dataloader, lossfun, device):
    net.eval()

    test_acc = []
    test_loss = []

    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device, dtype=torch.float)
            yHat = net(X)
            yHat = yHat.squeeze()
            yHat, y = yHat.cpu(), y.cpu()

            loss = lossfun(yHat, y)
            test_acc.append(torch.mean(((yHat > 0.0) == y).float()) * 100)
            test_loss.append(loss.item())

    # Calculate overall test accuracy and loss
    test_accuracy = np.mean(test_acc)
    test_mean_loss = np.mean(test_loss)

    return test_accuracy, test_mean_loss

# Kullanım örneği
test_accuracy, test_mean_loss = evaluateTest(net, test_dataloader, lossfun, device)
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Test Mean Loss: {test_mean_loss:.4f}")


In [ ]:
net.eval()

images = []
yTrue, yPred = [], []

with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device, dtype=torch.float)
        yHat = net(X)
        yHat = yHat.squeeze()
        yHat, y = yHat.cpu(), y.cpu()

        images.extend(X)
        yTrue.extend(y)
        yPred.extend((yHat > .0).float())

In [ ]:
yTrue, yPred = [yItem.item() for yItem in yTrue], [yItem.item() for yItem in yPred]


In [ ]:
def showComparision(images, yTrue, yPred):
    cols = 3
    rows = int(len(images) / cols)

    fig, ax = plt.subplots(rows, cols, figsize = (50, 200))
    fig.subplots_adjust(hspace=.5, wspace=.2)  # Adjust spacing between subplots

    for idx, image in enumerate(images):
        yActual = int(yTrue[idx])
        yPredicted = int(yPred[idx])

        yActualStr = test_dataset.classes[yActual]
        yPredictedStr = test_dataset.classes[yPredicted]

        correctPrediction = (yActual == yPredicted)
        color = 'green' if correctPrediction else 'red'

        i = int(idx / cols)
        j = int(idx % cols)

        # Normalize pixel values to range [0, 1]
        image = image.cpu().view(-1, imgSize, imgSize).permute(1, 2, 0)
        image = image.numpy()  # Convert tensor to NumPy array
        image = (image - np.min(image)) / (np.max(image) - np.min(image))  # Normalize

        # Add padding to the image
        pad = 0.1
        ax[i][j].imshow(image, aspect='auto')
        ax[i][j].set_title(f'Actual: {yActualStr} | Predicted: {yPredictedStr}', fontsize = 40, color = color, y = 1.05)
        ax[i][j].axis('off')

        border = Rectangle((0, 0), image.shape[1]-1, image.shape[0]-1, linewidth=5, edgecolor=color, facecolor='none')
        ax[i][j].add_patch(border)


In [ ]:
visualizableCount = 33
visualizableImages = images[:visualizableCount]
visualizableYTrue = yTrue[:visualizableCount]
visualizableYPred = yPred[:visualizableCount]
showComparision(visualizableImages, visualizableYTrue, visualizableYPred)

In [ ]:
cm = confusion_matrix(yTrue, yPred)
class_names = test_dataset.classes
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, cbar=False, xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
precision = precision_score(yTrue, yPred)
recall = recall_score(yTrue, yPred)
f1Score = f1_score(yTrue, yPred)

In [ ]:
from sklearn.metrics import classification_report
class_names = class_names  # Example class names

print(classification_report(yTrue, yPred, target_names=class_names))

In [ ]:
html_code = f"""
    <div style="display: flex; flex-direction: row; justify-content: center; align-items: center; gap: 30px">
        <span>
            <h3 style="line-height: 30px; background-color: #cc3d76; padding: 20px 20px; border-radius: 8px">Precision {precision:.2f}</h3>
        </span>

        <span>
            <h3 style="line-height: 30px; background-color: #b3245d; padding: 20px 20px; border-radius: 8px">Recall {recall:.2f}</h3>
        </span>

        <span>
            <h3 style="line-height: 30px; background-color: #d40457; padding: 20px 20px; border-radius: 8px">F1-Score {f1Score:.2f}</h3>
        </span>
    </div>
"""
HTML(html_code)